In [39]:
#import jax.numpy as np
#from jax import pmap
import numpy as np
from maxnorm.maxnorm_completion import *
from maxnorm.tenalg import *
from maxnorm.graphs import *
import sparse
from itertools import product
import networkx as nx

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# create random, low-rank tensor
t = 5
n = 30
r = 3
max_iterations = 100
epsilons = np.logspace(-4, 0, 5)
epsilons = np.concatenate((np.array(0), epsilons), axis = None)
#delta = 0.1
const = 24
sigma = 0.005
ndata =  const * r * t * n * np.log10(n)
U = kr_random(n, t, r, rvs='unif')
U = kr_rescale(U, np.sqrt(n**t), 'hs')
norm_true = np.sqrt(kr_dot(U, U) / n**t)
#U = [np.random.randn(n, r) for i in range(t)]
print("n data: %.2e" % ndata)
print("n data ** t/2: %.2e" % int(const * r * n**(t/2) * np.log10(n)))
print("true norm: %.2e" % norm_true)

n data: 1.60e+04
n data ** t/2: 5.24e+05
true norm: 1.00e+00


In [41]:
qnorm_true = max_qnorm_ub(U)
print(qnorm_true)
print(r**(t/2))

75.06169429017908
15.588457268119896


In [42]:
expander = nx.random_regular_graph(6, n)
#expander = nx.chordal_cycle_graph(n)
observation_mask = obs_mask_expander(expander, t)
#observation_mask = obs_mask_iid(tuple([n for i in range(t)]), ndata * n**(-t))

In [43]:
from run_sweep_iid import generate_data


data = generate_data(observation_mask, U, sigma)
clean_data_rmse = np.sqrt(loss(U, data) / data.nnz)
print(data.nnz)
print(ndata)
print(n**t)
print("%0.1e%%" % (float(data.nnz) / n**t * 100))

38880
15952.909550972354
24300000
1.6e-01%


In [44]:
def print_factor_norms(U):
    print("fro:   " + str([np.linalg.norm(Ui,'fro')**2 for Ui in U]))
    print("2-inf: " + str([np.max(np.linalg.norm(Ui, axis=1)) for Ui in U]))

print_factor_norms(kr_balance_factors(U))

fro:   [71.85212822731744, 71.85212822731744, 71.85212822731744, 71.85212822731744, 71.85212822731744]
2-inf: [2.483179533936205, 2.627452577192708, 2.108131154869586, 2.388633652367736, 2.2997527375367492]


In [45]:
delta = 1.5 * sigma
#np.sqrt(np.sum((data.data - clean_data)**2)) * 2 / np.sqrt(data.nnz) * 1.5

print("rms of data:        %f" % clean_data_rmse)
print("delta parameter:    %f" % delta)
print("rmse of U true:     %f" % np.sqrt(loss(U, data) / data.nnz))

rms of data:        0.004982
delta parameter:    0.007500
rmse of U true:     0.004982


In [46]:
#Uinit = kr_rescale(Unew1, np.sqrt(np.product(data.shape) * data.sum() ** 2 / data.nnz), 'hs')
# Uinit = kr_balance_factors(Unew1)

In [ ]:
%autoreload
Unew2_list = []
cost_arr_list = []
deltas = np.array([0, 1e-6, 1e-5, 1e-4, 1e-3, 2.5e-3])
eps = 1e-2
print(deltas)
print(sigma)
# sigma = deltas[-1] * np.sqrt(data.nnz)
for delta in deltas:
    print("delta = ", delta)
    Unew2, cost_arr = tensor_completion_maxnorm(data, 4 * r**t, delta, epsilon=eps,
#                                                 sgd=True, sgd_batch_size=2500,
                                                #U0 = kr_balance_factors(Unew1),
                                                init='svdrand',
                                                kappa=100, beta=1, 
                                                tol=1e-10, inner_tol=1e-12, max_iter=300, inner_max_iter=10, 
                                                verbosity=1, inner_line_iter=40,
                                                rebalance=True)
    Unew2_list.append(Unew2)
    cost_arr_list.append(cost_arr)


[0.0e+00 1.0e-06 1.0e-05 1.0e-04 1.0e-03 2.5e-03]
0.005
delta =  0.0
Initial cost: 1.967e+06
Initial qnorm_ub: 8.932e+03
|| r || = 1.004e+00, delta = 0.000e+00

Iteration 0 complete


scaled || r || = 2.949e-01, delta = 0.000e+00
Max-qnorm upper bound: 3.071e+05
Cost function:         4.763e+05



Iteration 1 complete


scaled || r || = 1.281e-01, delta = 0.000e+00
Max-qnorm upper bound: 1.827e+05
Cost function:         2.147e+05



Iteration 2 complete


scaled || r || = 9.416e-02, delta = 0.000e+00
Max-qnorm upper bound: 1.408e+05
Cost function:         1.581e+05



Iteration 3 complete


scaled || r || = 7.931e-02, delta = 0.000e+00
Max-qnorm upper bound: 1.157e+05
Cost function:         1.280e+05



Iteration 4 complete


scaled || r || = 6.978e-02, delta = 0.000e+00
Max-qnorm upper bound: 9.791e+04
Cost function:         1.074e+05




In [ ]:
qnorm_max = max_qnorm_ub(Unew2)
print(qnorm_max)

In [ ]:
expander_eigs = np.abs(nx.linalg.adjacency_spectrum(expander))
expander_eigs.sort()
print(expander_eigs[-10:])

In [ ]:
bound = 2**(2*t-4) * 1.8**(t-1) * (qnorm_max + qnorm_true)**2 * ((1 + expander_eigs[-2]/expander_eigs[-1])**(t-1) - 1)
print(bound)
print(bound / n**t)

In [ ]:
for i in range(len(Unew2_list)):
    print("delta = {}".format(deltas[i]))
    print("loss of U true:     %.3e" % np.sqrt(loss(U, data) / data.nnz))
    print("loss of U max:      %.3e\n" % np.sqrt(loss(Unew2_list[i], data) / data.nnz))

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(cost_arr_list[0])
plt.semilogy(cost_arr_list[1])
plt.semilogy(cost_arr_list[2])
plt.semilogy(cost_arr_list[3])
plt.semilogy(cost_arr_list[4])

plt.legend(["delta = 0", "delta = 1e-6", "delta = 1e-5", "delta = 1e-4", "delta = 1e-3", "delta = 2.5e-3"])
plt.xlabel('iterate')
plt.ylabel('cost')
plt.title("Cost-Curve for Various Delta Parameters, Sigma = 0.005")
plt.savefig('Cost-Curve-Delta.png')

## Generalization error

In [ ]:
def gen_err(Upred, Utrue):
    norm_true = kr_dot(Utrue, Utrue)
    mse_gen = kr_dot(Upred, Upred) + norm_true - 2 * kr_dot(Upred, Utrue)
    return np.sqrt(mse_gen / norm_true)

def mse_gen_err(Upred, Utrue):
    norm_true = kr_dot(Utrue, Utrue)
    mse_gen = kr_dot(Upred, Upred) + norm_true - 2 * kr_dot(Upred, Utrue)
    return np.sqrt(mse_gen / Upred[0].shape[0] ** len(Upred))


for i in range(len(Unew2_list)):
    print("delta = {}".format(deltas[i]))
    print("relative RMSE max:      %1.4e" % gen_err(Unew2_list[i], U))
    print("MSE max:                %1.4e\n" % mse_gen_err(Unew2_list[i], U))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1,t, figsize=(6, 20))
for i in range(t):
    im = axs[i].imshow(Unew2[i])
    divider = make_axes_locatable(axs[i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')
    if i > 0:
        axs[i].set_yticks([])

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1,t, figsize=(6, 20))
for i in range(t):
    im = axs[i].imshow(U[i])
    divider = make_axes_locatable(axs[i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')
    if i > 0:
        axs[i].set_yticks([])